**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# ⚠️ __work in progress__ not ready ⚠️

In [1]:
pwd

/minikube-host/notebooks/training-kubernetes-security


In [5]:
docker run openpolicyagent/opa version

Unable to find image 'openpolicyagent/opa:latest' locally
latest: Pulling from openpolicyagent/opa

4d09b7be: Pulling fs layer 
d1a13b69: Pulling fs layer 
879bb7de: Pulling fs layer 
Digest: sha256:785532b5ffa4073e24322c5d2d2067ba01ceed6a31934bb1c7f7af9d257dd415
Status: Downloaded newer image for openpolicyagent/opa:latest
Version: 0.35.0
Build Commit: a54537a
Build Timestamp: 2021-12-01T02:11:19Z
Build Hostname: 7e15c2339707
Go Version: go1.17.3
WebAssembly: available


In [3]:
mkdir opa
cd opa

In [4]:
cat > example.rego <<EOF
package example

greeting = msg {
    info := opa.runtime()
    hostname := info.env["HOSTNAME"] # Kubernetes sets the HOSTNAME environment variable.
    msg := sprintf("hello from pod %q!", [hostname])
}
EOF

In [21]:
kubectl create configmap example-policy --from-file example.rego

configmap/example-policy created


In [24]:
kubectl create -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: opa
  labels:
    app: opa
spec:
  replicas: 1
  selector:
    matchLabels:
      app: opa
  template:
    metadata:
      labels:
        app: opa
      name: opa
    spec:
      containers:
      - name: opa
        image: openpolicyagent/opa:0.35.0
        ports:
        - name: http
          containerPort: 8181
        args:
        - "run"
        - "--ignore=.*"  # exclude hidden dirs created by Kubernetes
        - "--server"
        - "/policies"
        volumeMounts:
        - readOnly: true
          mountPath: /policies
          name: example-policy
      volumes:
      - name: example-policy
        configMap:
          name: example-policy
EOF

deployment.apps/opa created


In [8]:
kubectl create -f - <<EOF
kind: Service
apiVersion: v1
metadata:
  name: opa
  labels:
    app: opa
spec:
  type: NodePort
  selector:
    app: opa
  ports:
    - name: http
      protocol: TCP
      port: 8181
      targetPort: 8181
EOF

service/opa created


In [11]:
kubectl get svc opa -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2021-12-03T09:27:52Z"
  labels:
    app: opa
  name: opa
  namespace: default
  resourceVersion: "1063"
  uid: 04b7e10c-9b8c-4641-a2e0-1027a898f430
spec:
  clusterIP: 10.96.83.229
  clusterIPs:
  - 10.96.83.229
  externalTrafficPolicy: Cluster
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - name: http
    nodePort: 32505
    port: 8181
    protocol: TCP
    targetPort: 8181
  selector:
    app: opa
  sessionAffinity: None
  type: NodePort
status:
  loadBalancer: {}


In [39]:
OPA_IP_PORT=$(hostname -i):$(kubectl get service opa -o jsonpath='{.spec.ports[0].nodePort}')
echo $OPA_IP_PORT

192.168.49.2:32505


In [40]:
curl $OPA_IP_PORT/v1/data

{"result":{"example":{"greeting":"hello from pod \"opa-69c9967959-gf4wh\"!"}}}

In [41]:
kubectl get all

NAME                       READY   STATUS    RESTARTS   AGE
pod/opa-69c9967959-gf4wh   1/1     Running   0          7m43s

NAME                 TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE
service/kubernetes   ClusterIP   10.96.0.1      <none>        443/TCP          24m
service/opa          NodePort    10.96.83.229   <none>        8181:32505/TCP   14m

NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/opa   1/1     1            1           7m43s

NAME                             DESIRED   CURRENT   READY   AGE
replicaset.apps/opa-69c9967959   1         1         1       7m43s


In [42]:
kubectl get events

LAST SEEN   TYPE      REASON                    OBJECT                      MESSAGE
24m         Normal    Starting                  node/minikube               Starting kubelet.
24m         Normal    NodeHasSufficientMemory   node/minikube               Node minikube status is now: NodeHasSufficientMemory
24m         Normal    NodeHasNoDiskPressure     node/minikube               Node minikube status is now: NodeHasNoDiskPressure
24m         Normal    NodeHasSufficientPID      node/minikube               Node minikube status is now: NodeHasSufficientPID
24m         Normal    NodeAllocatableEnforced   node/minikube               Updated Node Allocatable limit across pods
24m         Normal    NodeReady                 node/minikube               Node minikube status is now: NodeReady
24m         Normal    RegisteredNode            node/minikube               Node minikube event: Registered Node minikube in Controller
7m51s       Normal    Scheduled                 pod/opa-69c9967959-gf4

In [28]:
minikube service list

bash: minikube: command not found


: 127